Import Libraries

In [1]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

Read Data

In [2]:
df_train = pd.read_csv("2021_stats.csv", delimiter=",")
df_train[0:5]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,isAllNBA
0,1,Julius Randle\randlju01,PF,26,NYK,71,71,37.6,8.5,18.6,...,1.2,9.0,10.2,6.0,0.9,0.3,3.4,3.2,24.1,1
1,2,James Harden\hardeja01,PG-SG,31,TOT,44,43,36.6,7.8,16.7,...,0.8,7.1,7.9,10.8,1.2,0.8,4.0,2.3,24.6,0
2,3,Fred VanVleet\vanvlfr01,SG,26,TOR,52,52,36.5,6.5,16.7,...,0.6,3.6,4.2,6.3,1.7,0.7,1.8,2.3,19.6,0
3,4,Russell Westbrook\westbru01,PG,32,WAS,65,65,36.4,8.4,19.0,...,1.7,9.9,11.5,11.7,1.4,0.4,4.8,2.9,22.2,0
4,5,Harrison Barnes\barneha02,PF,28,SAC,58,58,36.2,5.5,11.1,...,1.1,5.5,6.6,3.5,0.7,0.2,1.6,1.3,16.1,0


In [3]:
df_train.corr(method='pearson')

,Rk,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,isAllNBA
Rk,1.000000,0.017067,-0.126097,-0.371354,-0.995996,-0.714761,-0.732874,-0.038744,-0.320328,-0.332744,...,0.062413,-0.247809,-0.165063,-0.569266,-0.278964,0.092101,-0.618547,-0.120082,-0.740098,-0.313829
Age,0.017067,1.000000,0.004369,0.024457,-0.015140,-0.032720,-0.041384,-0.016677,0.176386,0.139840,...,-0.141679,0.057028,-0.003176,0.147035,0.111272,-0.004380,-0.070639,-0.133953,0.003414,0.212730
G,-0.126097,0.004369,1.000000,0.796880,0.121302,-0.046171,-0.072497,0.035126,0.091617,0.009884,...,0.029449,0.140389,0.116441,-0.012050,0.009948,-0.091438,-0.067117,-0.128132,-0.030145,0.117923
GS,-0.371354,0.024457,0.796880,1.000000,0.361508,0.175425,0.127234,0.135218,0.038048,-0.029263,...,0.144033,0.249078,0.235765,0.108501,0.104728,0.012574,0.112723,0.045090,0.176307,0.205531
MP,-0.995996,-0.015140,0.121302,0.361508,1.000000,0.709546,0.729553,0.036101,0.318190,0.331939,...,-0.057388,0.262079,0.177236,0.569216,0.273199,-0.088364,0.618766,0.135518,0.735800,0.317107
FG,-0.714761,-0.032720,-0.046171,0.175425,0.709546,1.000000,0.939273,0.284099,0.246937,0.248874,...,0.105062,0.347860,0.295493,0.482605,0.123458,0.020650,0.673621,0.110704,0.975758,0.460953
FGA,-0.732874,-0.041384,-0.072497,0.127234,0.729553,0.939273,1.000000,-0.043008,0.428398,0.467462,...,-0.124528,0.181452,0.094109,0.542096,0.166942,-0.138551,0.695673,0.028603,0.953688,0.384679
FG%,-0.038744,-0.016677,0.035126,0.135218,0.036101,0.284099,-0.043008,1.000000,-0.479511,-0.578227,...,0.709691,0.493480,0.606337,-0.109934,-0.127746,0.484061,0.025778,0.234975,0.173320,0.230692
3P,-0.320328,0.176386,0.091617,0.038048,0.318190,0.246937,0.428398,-0.479511,1.000000,0.975777,...,-0.603048,-0.294479,-0.421857,0.187044,0.034383,-0.354079,0.115857,-0.269821,0.356847,0.105553
3PA,-0.332744,0.139840,0.009884,-0.029263,0.331939,0.248874,0.467462,-0.578227,0.975777,1.000000,...,-0.636985,-0.321835,-0.453689,0.220099,0.042422,-0.363411,0.168149,-0.234856,0.364555,0.090599


In [4]:
df_test = pd.read_csv("2022_stats.csv", delimiter=",")
df_test[0:5]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Fred VanVleet\vanvlfr01,PG,27,TOR,27,27,38.0,7.0,16.2,...,0.864,0.7,4.3,5.0,6.5,1.6,0.6,3.1,2.6,19.8
1,2,OG Anunoby\anunoog01,SF,24,TOR,15,15,37.3,7.6,17.7,...,0.825,1.6,3.8,5.4,2.7,1.5,0.6,1.7,2.7,20.1
2,3,LeBron James\jamesle01,SF,37,LAL,17,17,37.1,9.9,19.7,...,0.763,0.6,5.3,5.9,6.9,1.8,0.9,3.6,2.1,26.1
3,4,Kevin Durant\duranke01,PF,33,BRK,26,26,36.9,10.7,20.5,...,0.882,0.5,7.3,7.8,5.8,0.7,0.8,3.2,1.8,29.6
4,5,Miles Bridges\bridgmi02,PF,23,CHO,30,30,36.6,7.4,15.7,...,0.760,1.3,6.0,7.3,3.6,1.1,0.9,1.9,2.7,19.8


Pre-processing

In [5]:
df_train = df_train.drop(labels=['Rk','Player','Pos','Tm','Age','G','GS','3P','3PA','3P%','FT%','ORB','BLK','PF'],axis=1)
df_train = df_train.fillna(0)
df_train[0:5]

,MP,FG,FGA,FG%,2P,2PA,2P%,eFG%,FT,FTA,DRB,TRB,AST,STL,TOV,PTS,isAllNBA
0,37.6,8.5,18.6,0.456,6.2,13.1,0.474,0.516,4.9,6.0,9.0,10.2,6.0,0.9,3.4,24.1,1
1,36.6,7.8,16.7,0.466,5.0,9.1,0.553,0.548,6.3,7.3,7.1,7.9,10.8,1.2,4.0,24.6,0
2,36.5,6.5,16.7,0.389,3.2,7.6,0.416,0.489,3.3,3.7,3.6,4.2,6.3,1.7,1.8,19.6,0
3,36.4,8.4,19.0,0.439,7.0,14.8,0.475,0.474,4.2,6.4,9.9,11.5,11.7,1.4,4.8,22.2,0
4,36.2,5.5,11.1,0.497,3.8,6.7,0.567,0.575,3.4,4.1,5.5,6.6,3.5,0.7,1.6,16.1,0


In [6]:
df_test = df_test.drop(labels=['Rk','Player','Pos','Tm','Age','G','GS','3P','3PA','3P%','FT%','ORB','BLK','PF'],axis=1)
df_test = df_test.fillna(0)
df_test[0:5]

,MP,FG,FGA,FG%,2P,2PA,2P%,eFG%,FT,FTA,DRB,TRB,AST,STL,TOV,PTS
0,38.0,7.0,16.2,0.430,3.7,7.7,0.476,0.532,2.6,3.0,4.3,5.0,6.5,1.6,3.1,19.8
1,37.3,7.6,17.7,0.430,4.9,10.2,0.477,0.508,2.2,2.7,3.8,5.4,2.7,1.5,1.7,20.1
2,37.1,9.9,19.7,0.504,7.1,11.8,0.602,0.576,3.4,4.5,5.3,5.9,6.9,1.8,3.6,26.1
3,36.9,10.7,20.5,0.523,8.8,15.5,0.569,0.568,6.3,7.2,7.3,7.8,5.8,0.7,3.2,29.6
4,36.6,7.4,15.7,0.470,5.4,9.6,0.561,0.534,3.1,4.0,6.0,7.3,3.6,1.1,1.9,19.8


In [7]:
X_train = df_train[['MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'DRB', 'TRB', 'AST', 'STL', 'TOV', 'PTS']].values
le_pos = preprocessing.LabelEncoder()
X_train[0:5]

array([[37.6  ,  8.5  , 18.6  ,  0.456,  6.2  , 13.1  ,  0.474,  0.516,
         4.9  ,  6.   ,  9.   , 10.2  ,  6.   ,  0.9  ,  3.4  , 24.1  ],
       [36.6  ,  7.8  , 16.7  ,  0.466,  5.   ,  9.1  ,  0.553,  0.548,
         6.3  ,  7.3  ,  7.1  ,  7.9  , 10.8  ,  1.2  ,  4.   , 24.6  ],
       [36.5  ,  6.5  , 16.7  ,  0.389,  3.2  ,  7.6  ,  0.416,  0.489,
         3.3  ,  3.7  ,  3.6  ,  4.2  ,  6.3  ,  1.7  ,  1.8  , 19.6  ],
       [36.4  ,  8.4  , 19.   ,  0.439,  7.   , 14.8  ,  0.475,  0.474,
         4.2  ,  6.4  ,  9.9  , 11.5  , 11.7  ,  1.4  ,  4.8  , 22.2  ],
       [36.2  ,  5.5  , 11.1  ,  0.497,  3.8  ,  6.7  ,  0.567,  0.575,
         3.4  ,  4.1  ,  5.5  ,  6.6  ,  3.5  ,  0.7  ,  1.6  , 16.1  ]])

In [8]:
y_train = df_train["isAllNBA"]
y_train [0:5]

0    1
1    0
2    0
3    0
4    0
Name: isAllNBA, dtype: int64

In [10]:
X_test = df_test[['MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'DRB', 'TRB', 'AST', 'STL', 'TOV', 'PTS']].values
le_pos2 = preprocessing.LabelEncoder()
X_test[0:5]

array([[38.   ,  7.   , 16.2  ,  0.43 ,  3.7  ,  7.7  ,  0.476,  0.532,
         2.6  ,  3.   ,  4.3  ,  5.   ,  6.5  ,  1.6  ,  3.1  , 19.8  ],
       [37.3  ,  7.6  , 17.7  ,  0.43 ,  4.9  , 10.2  ,  0.477,  0.508,
         2.2  ,  2.7  ,  3.8  ,  5.4  ,  2.7  ,  1.5  ,  1.7  , 20.1  ],
       [37.1  ,  9.9  , 19.7  ,  0.504,  7.1  , 11.8  ,  0.602,  0.576,
         3.4  ,  4.5  ,  5.3  ,  5.9  ,  6.9  ,  1.8  ,  3.6  , 26.1  ],
       [36.9  , 10.7  , 20.5  ,  0.523,  8.8  , 15.5  ,  0.569,  0.568,
         6.3  ,  7.2  ,  7.3  ,  7.8  ,  5.8  ,  0.7  ,  3.2  , 29.6  ],
       [36.6  ,  7.4  , 15.7  ,  0.47 ,  5.4  ,  9.6  ,  0.561,  0.534,
         3.1  ,  4.   ,  6.   ,  7.3  ,  3.6  ,  1.1  ,  1.9  , 19.8  ]])

In [ ]:
y_test = df_test["isAllNBA"]
y_test [0:5]

In [11]:
sm = SMOTE(random_state = 42)
X_res, y_res = sm.fit_resample(X_train, y_train)

Modeling

In [12]:
decTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4, random_state = 42)
decTree # it shows the default parameters

DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=42)

In [13]:
decTree.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=42)

Prediction

In [14]:
predTree = decTree.predict(X_test)

In [22]:
predTree[0:150]

array([0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

Evaluation

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, predTree))

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, predTree, average='weighted') 

In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(y_test, predTree,pos_label=0)

Visualization

In [ ]:
from  io import StringIO
import pydotplus
import matplotlib.image as mpimg
from sklearn import tree
%matplotlib inline 

In [ ]:
dot_data = StringIO()
filename = "dectree.png"
featureNames = df_test.columns[0:17]
out=tree.export_graphviz(decTree,feature_names=featureNames, out_file=dot_data, class_names= ["0","1"], filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, predTree, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, predTree))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not All-NBA(0)','All-NBA(1)'],normalize= False,  title='Confusion matrix')

In [18]:
decTree2 = DecisionTreeClassifier(criterion="entropy", max_depth = 4, random_state = 42)
decTree2.fit(X_res,y_res)
predTree2 = decTree2.predict(X_test)

In [23]:
predTree2[0:150]

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
f1_score(y_test, predTree2, average='weighted') 

In [ ]:
jaccard_score(y_test, predTree2,pos_label=0)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, predTree2, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, predTree2))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not All-NBA(0)','All-NBA(1)'],normalize= False,  title='Confusion matrix')

In [ ]:
dot_data = StringIO()
filename = "dectree.png"
featureNames = df_test.columns[0:17]
out=tree.export_graphviz(decTree2,feature_names=featureNames, out_file=dot_data, class_names= ["0","1"], filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')

In [25]:
decTree3 = DecisionTreeClassifier(criterion="gini", max_depth = 4, random_state = 42)
decTree3.fit(X_train,y_train)
predTree3 = decTree3.predict(X_test)

In [26]:
predTree3[0:150]

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
f1_score(y_test, predTree3, average='weighted') 

In [ ]:
jaccard_score(y_test, predTree3,pos_label=0)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, predTree3, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, predTree3))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not All-NBA(0)','All-NBA(1)'],normalize= False,  title='Confusion matrix')

In [ ]:
dot_data = StringIO()
filename = "dectree.png"
featureNames = df_test.columns[0:17]
out=tree.export_graphviz(decTree3,feature_names=featureNames, out_file=dot_data, class_names= ["0","1"], filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')

In [27]:
decTree4 = DecisionTreeClassifier(criterion="gini", max_depth = 4, random_state = 42)
decTree4.fit(X_res,y_res)
predTree4 = decTree4.predict(X_test)

In [28]:
predTree4[0:150]

array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
f1_score(y_test, predTree4, average='weighted') 

In [ ]:
jaccard_score(y_test, predTree4,pos_label=0)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, predTree4, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, predTree3))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not All-NBA(0)','All-NBA(1)'],normalize= False,  title='Confusion matrix')

In [ ]:
dot_data = StringIO()
filename = "dectree.png"
featureNames = df_test.columns[0:17]
out=tree.export_graphviz(decTree4,feature_names=featureNames, out_file=dot_data, class_names= ["0","1"], filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')

In [29]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train) 

SVC()

In [30]:
yhat = clf.predict(X_test)
yhat [0:5]

array([0, 0, 0, 0, 0], dtype=int64)

In [31]:
yhat[0:150]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, yhat)

In [ ]:
f1_score(y_test, yhat, average='weighted') 

In [ ]:
jaccard_score(y_test, yhat,pos_label=0)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhat, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, yhat))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not All-NBA(0)','All-NBA(1)'],normalize= False,  title='Confusion matrix')

In [32]:
clf2 = svm.SVC(kernel='rbf')
clf2.fit(X_res, y_res) 

SVC()

In [33]:
yhat2 = clf2.predict(X_test)
yhat2 [0:5]

array([0, 0, 1, 1, 0], dtype=int64)

In [34]:
yhat2[0:150]

array([0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
accuracy_score(y_test, yhat2)

In [ ]:
f1_score(y_test, yhat2, average='weighted') 

In [ ]:
jaccard_score(y_test, yhat2,pos_label=0)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhat2, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, yhat2))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not All-NBA(0)','All-NBA(1)'],normalize= False,  title='Confusion matrix')

In [35]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
pipelineSVC = make_pipeline(StandardScaler(), SVC(random_state=1))
#
# Create the parameter grid
#
param_grid_svc = [{
                    'svc__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
                    'svc__kernel': ['linear']
                  },
                 {
                    'svc__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
                    'svc__gamma': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
                    'svc__kernel': ['rbf']
                 }]
#
# Create an instance of GridSearch Cross-validation estimator
#
gsSVC = GridSearchCV(estimator=pipelineSVC,
                     param_grid = param_grid_svc,
                     scoring='accuracy',
                     cv=10,
                     refit=True,
                     n_jobs=1)
#
# Train the SVM classifier
#
gsSVC.fit(X_train, y_train)
#
# Print the training score of the best model
#
print(gsSVC.best_score_)
#
# Print the model parameters of the best model
#
print(gsSVC.best_params_)

0.9133333333333333
{'svc__C': 10.0, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}


In [36]:
yhat3 = gsSVC.predict(X_test)
yhat3 [0:5]

array([0, 0, 1, 1, 0], dtype=int64)

In [37]:
yhat3[0:150]

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
accuracy_score(y_test, yhat3)

In [ ]:
f1_score(y_test, yhat3, average='weighted') 

In [ ]:
jaccard_score(y_test, yhat3,pos_label=0)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhat3, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, yhat3))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not All-NBA(0)','All-NBA(1)'],normalize= False,  title='Confusion matrix')

In [38]:
gsSVC2 = GridSearchCV(estimator=pipelineSVC,
                     param_grid = param_grid_svc,
                     scoring='accuracy',
                     cv=10,
                     refit=True,
                     n_jobs=1)
#
# Train the SVM classifier
#
gsSVC2.fit(X_res, y_res)
#
# Print the training score of the best model
#
print(gsSVC2.best_score_)
#
# Print the model parameters of the best model
#
print(gsSVC2.best_params_)

0.9814814814814815
{'svc__C': 1.0, 'svc__gamma': 0.5, 'svc__kernel': 'rbf'}


In [39]:
yhat4 = gsSVC2.predict(X_test)
yhat4 [0:5]

array([0, 0, 0, 0, 0], dtype=int64)

In [40]:
yhat4[0:150]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
accuracy_score(y_test, yhat4)

In [ ]:
f1_score(y_test, yhat4, average='weighted') 

In [ ]:
jaccard_score(y_test, yhat4,pos_label=0)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhat4, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, yhat4))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not All-NBA(0)','All-NBA(1)'],normalize= False,  title='Confusion matrix')

In [41]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(64, ), activation='relu',max_iter=1000, epsilon=1e-08, random_state = 42)

In [42]:
from sklearn.metrics import accuracy_score
mlp.fit(X_train, y_train)
predANN = mlp.predict(X_test)

D:\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [43]:
predANN[0:150]

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
f1_score(y_test, predANN, average='weighted') 

In [ ]:
jaccard_score(y_test, predANN,pos_label=0)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, predANN, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, predANN))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not All-NBA(0)','All-NBA(1)'],normalize= False,  title='Confusion matrix')

In [45]:
mlp2 = MLPClassifier(hidden_layer_sizes=(64, ), activation='relu',max_iter=1000, epsilon=1e-08, random_state = 42)
mlp2.fit(X_res, y_res)
predANN2 = mlp2.predict(X_test)

In [46]:
predANN2[0:150]

array([0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
f1_score(y_test, predANN2, average='weighted') 

In [ ]:
jaccard_score(y_test, predANN2,pos_label=0)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, predANN2, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, predANN2))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not All-NBA(0)','All-NBA(1)'],normalize= False,  title='Confusion matrix')

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
predLR = LR.predict(X_test)

In [48]:
predLR[0:150]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
f1_score(y_test, predLR, average='weighted') 

In [ ]:
jaccard_score(y_test, predLR,pos_label=0)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, predLR, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, predLR))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not All-NBA(0)','All-NBA(1)'],normalize= False,  title='Confusion matrix')

In [49]:
LR2 = LogisticRegression(C=0.01, solver='liblinear').fit(X_res,y_res)
predLR2 = LR2.predict(X_test)

In [50]:
predLR2[0:150]

array([0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
f1_score(y_test, predLR2, average='weighted') 

In [ ]:
jaccard_score(y_test, predLR,pos_label=0)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, predLR2, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, predLR2))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not All-NBA(0)','All-NBA(1)'],normalize= False,  title='Confusion matrix')

In [51]:
pipelineLR = make_pipeline(StandardScaler(), LogisticRegression(random_state=1, penalty='l2', solver='lbfgs'))
#
# Create the parameter grid
#
param_grid_lr = [{
    'logisticregression__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
}]
#
# Create an instance of GridSearch Cross-validation estimator
#
gsLR = GridSearchCV(estimator=pipelineLR,
                     param_grid = param_grid_lr,
                     scoring='accuracy',
                     cv=10,
                     refit=True,
                     n_jobs=1)
#
# Train the LogisticRegression Classifier
#
gsLR = gsLR.fit(X_train, y_train)
#
# Print the training score of the best model
#
print(gsLR.best_score_)
#
# Print the model parameters of the best model
#
print(gsLR.best_params_)

0.9133333333333333
{'logisticregression__C': 0.5}


In [52]:
predgsLR = gsLR.predict(X_test)

In [53]:
predgsLR[0:150]

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
f1_score(y_test, predgsLR, average='weighted') 

In [ ]:
jaccard_score(y_test, predgsLR,pos_label=0)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, predgsLR, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, predgsLR))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not All-NBA(0)','All-NBA(1)'],normalize= False,  title='Confusion matrix')

In [54]:
pipelineLR = make_pipeline(StandardScaler(), LogisticRegression(random_state=1, penalty='l2', solver='lbfgs'))
#
# Create the parameter grid
#
param_grid_lr = [{
    'logisticregression__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
}]
#
# Create an instance of GridSearch Cross-validation estimator
#
gsLR2 = GridSearchCV(estimator=pipelineLR,
                     param_grid = param_grid_lr,
                     scoring='accuracy',
                     cv=10,
                     refit=True,
                     n_jobs=1)
#
# Train the LogisticRegression Classifier
#
gsLR2 = gsLR2.fit(X_res, y_res)
#
# Print the training score of the best model
#
print(gsLR2.best_score_)
#
# Print the model parameters of the best model
#
print(gsLR2.best_params_)

0.9444444444444444
{'logisticregression__C': 0.5}


In [55]:
predgsLR2 = gsLR2.predict(X_test)

In [56]:
predgsLR2[0:150]

array([1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
f1_score(y_test, predgsLR2, average='weighted') 

In [ ]:
jaccard_score(y_test, predgsLR2,pos_label=0)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, predgsLR2, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, predgsLR2))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Not All-NBA(0)','All-NBA(1)'],normalize= False,  title='Confusion matrix')